In [ ]:
from LoopStructural import GeologicalModel
from LoopStructural.modelling.fault.fault_function import BaseFault
from LoopStructural.visualisation import LavaVuModelViewer, MapView
from LoopStructural.utils import process_map2loop
import pandas as pd
import numpy as np
import os

## Create a model from map2loop
[map2loop](https://github.com/Loop3D/map2loop) is a map deconstruction library to provide inputs to [Loop](Loop3d.org). The development of map2loop is lead by Mark Jessell at The University of Western Australia. 

###  What it does:
Combines information extracted from vector geology maps in various forms to support 3D geological modelling. Outputs are simple csv files that should be readable by any 3D modelling system. In the following demonstration we will use LoopStructural to generate a 3D model automatically from a map2loop output. 

In [ ]:
fault_params = {'interpolatortype':'FDI',
                'nelements':1e4, #number of elements to model fault with
                'data_region':.5, #buffer around fault volume
                'solver':'pyamg', #how to solve least squares system, pyamg is an algorithmic multigrid solver
               }

foliation_params = {'nelements':1e5,# how many tetras/voxels
                    'interpolatortype':'PLI',
                    'buffer':.5,  # how much to extend nterpolation around box np.array([1000,1000,1000]),#
                    'solver':'pyamg',
                    'damp':True,
                   }

model, m2l_data = GeologicalModel.from_map2loop_directory('./data/scratch/',#'/home/lgrose/dev/python/map2loop/Turner_Syncline/',
                                                          skip_faults=False,
                                                          fault_params=fault_params,
#                                                           unconformities=False,
                                                          rescale=True,
                                                          foliation_params=foliation_params)



## Fault network
The following figure shows the fault displacement magnitude within the model area. Fault displacements are estimated from map2loop and geometry is constrained by the map2loop process, the faults are assumed to be vertical with vertical displacement vectors.  

In [ ]:
mapview = MapView(model)
mapview.nsteps = (200,200)
mapview.add_fault_displacements()
mapview.add_faults()

# for i in range(5):
#     def mask(x):
#         val = model.features[i].displacementfeature.evaluate_value(x)
#         val[np.isnan(val)] = 0
#         maskv = np.zeros(val.shape).astype(bool)
# #         maskv[~np.isnan(val)][np.abs(val[~np.isnan(val)]) > 0.001] = 1
#         maskv[np.abs(val) > 0.001] = 1
# #         maskv = lambda x : np.abs() > 0.01
#         return maskv
# #     mask = lambda x : np.abs(model.features[i].displacementfeature.evaluate_value(x)) > 0.0001
#     mapview.add_contour(model.features[i],0,mask=mask)

## Visualise 3D block diagram and boundary surfaces
A stratigraphic column is generated from the map2loop input data that is populated with the colours from the geological map. 

In [ ]:
# %%notify
view = LavaVuModelViewer(model,vertical_exaggeration=1)
view.nelements=1e6
view.add_model()
view.nelements=1e5
view.add_model_surfaces()
view.interactive()

### Plot on a map surface
We can also add the model to a 2D map (using a z value of 0, although this can be varied by adding `z` as an argument to the `add_model`

In [ ]:
mapview = MapView(model)
mapview.nsteps = (200,200)
mapview.add_faults()
mapview.add_model()
# mapview.add_data(model.get_feature_by_name('supergroup_0'),unfault=True,grad=False)